In [9]:
import pandas as pd
import tkinter as tk
import unicodedata
import io
import requests

# Função para determinar o delimitador do arquivo CSV
def determinar_delimitador(content):
    delimiters = [',', ';', '\t']
    for delimiter in delimiters:
        if delimiter in content:
            return delimiter
    return ','  # Delimitador padrão é a vírgula

# Função para remover acentos e caracteres especiais
def remove_accents(text):
    text = ''.join(char for char in unicodedata.normalize('NFD', text) if unicodedata.category(char) != 'Mn')
    return text

# Função para baixar, limpar e filtrar o arquivo CSV
def download_clean_filter(url, selected_columns):
    try:
        # Realiza a solicitação HTTP para baixar o arquivo CSV
        response = requests.get(url)
        response.raise_for_status()

        # Lê o conteúdo do arquivo CSV e determina o delimitador
        content = response.content.decode('utf-8')
        delimiter = determinar_delimitador(content)

        # Lê o arquivo CSV ajustando os parâmetros
        df = pd.read_csv(io.StringIO(content), delimiter=';', quotechar='"', escapechar='\\', na_values=["", "None"])

        # Remove acentos e caracteres especiais dos rótulos das colunas
        df.columns = [remove_accents(col).upper() for col in df.columns]

        # Verifica se as colunas especificadas existem no DataFrame
        selected_columns = [col for col in selected_columns if col in df.columns]

        if not selected_columns:
            selected_columns = df.columns[:4]

        # Aplica os filtros nas colunas selecionadas
        filtered_df = df[selected_columns]

        return filtered_df
    except Exception as e:
        return str(e)  # Retorna a mensagem de erro

# Função para lidar com o botão "Download"
def download_button_clicked():
    url = entry_url.get()
    selected_columns = entry_columns.get().split(',')
    result = download_clean_filter(url, selected_columns)

    if isinstance(result, pd.DataFrame):
        # Exibir o DataFrame filtrado
        result_label.config(text="Arquivo CSV baixado, limpo e filtrado com sucesso.")
        result_text.delete(1.0, tk.END)
        result_text.insert(tk.END, result.to_string(index=False))
    else:
        result_label.config(text="Ocorreu um erro com arquivo CSV")
        print(result)

# Configuração da janela principal
window = tk.Tk()
window.title("Aplicativo de Leitura, Limpeza e Filtro de Arquivo CSV")

# Rótulo e entrada para a URL do arquivo CSV
url_label = tk.Label(window, text="URL do arquivo CSV:")
url_label.pack()
entry_url = tk.Entry(window, width=50)
entry_url.pack()

# Rótulo e entrada para os IDs das colunas de filtro
columns_label = tk.Label(window, text="IDs das colunas (separados por vírgula):")
columns_label.pack()
entry_columns = tk.Entry(window, width=50)
entry_columns.pack()

# Botão para baixar, limpar e filtrar o arquivo CSV
download_button = tk.Button(window, text="Baixar, Limpar e Filtrar", command=download_button_clicked)
download_button.pack()

# Rótulo e caixa de texto para exibir o resultado
result_label = tk.Label(window, text="")
result_label.pack()
result_text = tk.Text(window, height=10, width=50)
result_text.pack()

# Iniciar a interface gráfica
window.mainloop()